## Strategia

In [1]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
import numpy as np
import pandas as pd


#wyceniam opcje za pomoca Black Scholesa
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

In [2]:
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S*bs_call(S,K,T,r,sigma)

In [3]:
df = pd.read_csv("cocoa.csv")
df.dropna()
df = df.assign(close_day_before=df.Close.shift(1))
df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)

In [4]:
sigma = np.sqrt(252) * df['returns'].std() 

In [25]:
lcp = df['Close'].iloc[-1] #ostatnia cena, 
t = 90/365 #data wygasniecia , 3 m-ce
strike_price = 2442
r = 0.06 #r wolna od ryzyka

In [26]:
prem = bs_call(lcp, strike_price, t, 0.05, sigma) #robimy long calla (chyba)
prem

140.49864685914895

In [27]:
cena_o = prem + strike_price
cena_o

2582.4986468591487

In [28]:
ceny = np.arange(-500,500,1) + lcp #hipotetyczne ceny, zmiennosc co 1 dolar
#ceny

In [29]:
cena_opcji = []
for i in ceny:
    if i >= cena_o:
        cena_opcji.append(cena_o)
    else:
        cena_opcji.append(i+prem)

In [30]:
cena_opcji[:10]

[2082.4986468591487,
 2083.4986468591487,
 2084.4986468591487,
 2085.4986468591487,
 2086.4986468591487,
 2087.4986468591487,
 2088.4986468591487,
 2089.4986468591487,
 2090.4986468591487,
 2091.4986468591487]

In [31]:
d = {'Cena kakao':ceny,'cena opcji':cena_opcji}
porownanie = pd.DataFrame(d)
porownanie.iloc[495:505]

,Cena kakao,cena opcji
495,2437.0,2577.498647
496,2438.0,2578.498647
497,2439.0,2579.498647
498,2440.0,2580.498647
499,2441.0,2581.498647
500,2442.0,2582.498647
501,2443.0,2583.498647
502,2444.0,2584.498647
503,2445.0,2585.498647
504,2446.0,2586.498647


In [32]:
porownanie["zysk/strata"] = porownanie["Cena kakao"]-porownanie["cena opcji"]

In [33]:
porownanie.iloc[495:505]

,Cena kakao,cena opcji,zysk/strata
495,2437.0,2577.498647,-140.498647
496,2438.0,2578.498647,-140.498647
497,2439.0,2579.498647,-140.498647
498,2440.0,2580.498647,-140.498647
499,2441.0,2581.498647,-140.498647
500,2442.0,2582.498647,-140.498647
501,2443.0,2583.498647,-140.498647
502,2444.0,2584.498647,-140.498647
503,2445.0,2585.498647,-140.498647
504,2446.0,2586.498647,-140.498647


In [34]:
porownanie["zysk/strata w procencie"] = porownanie["zysk/strata"]/porownanie["Cena kakao"]#nie wiem czy tak trzeba
porownanie

,Cena kakao,cena opcji,zysk/strata,zysk/strata w procencie
0,1942.0,2082.498647,-140.498647,-0.072347
1,1943.0,2083.498647,-140.498647,-0.072310
2,1944.0,2084.498647,-140.498647,-0.072273
3,1945.0,2085.498647,-140.498647,-0.072236
4,1946.0,2086.498647,-140.498647,-0.072199
...,...,...,...,...
995,2937.0,2582.498647,354.501353,0.120702
996,2938.0,2582.498647,355.501353,0.121001
997,2939.0,2582.498647,356.501353,0.121300
998,2940.0,2582.498647,357.501353,0.121599


## Obliczanie wartości zagrożonej

### Meoda historyczna zwykła

In [44]:
np.quantile(-porownanie["zysk/strata w procencie"], 0.95)

0.07053322048336692

In [45]:
np.quantile(-porownanie["zysk/strata w procencie"], 0.99)

0.07197713473141867

In [41]:
np.quantile(porownanie["zysk/strata w procencie"], 0.05)

-0.07053322048336692

In [42]:
np.quantile(porownanie["zysk/strata w procencie"], 0.01)

-0.07197713473141867